# SQL Business Analytics - Chinook

In this exercise we will be answering some hypothetical questions about a hypothetical online music store (similar to iTunes) called Chinook. We will be answering questions related to Chinook's business, employee performance, etc. We will also perform a bit of data segmentation.

## 1 - Importing the data

First let's setup the database that we'll use to answer these questions.

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view")

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Looks like the database was successfully imported. The database comprises of 10 tables containing all kinds of information--from Chinook's employee roster, to their entire customer roster. Below is an overview of the information contained in each table, as well as the key columns connecting different tables:

<img src="chinook_database_schema.png" width="600" height="400">

## 2 - Data Analysis

Let's begin answering some questions with this database.

### 2.1 - Which genre performs the best in the USA?
The Chinook store has just signed a deal with a record label. The deal is such that Chinook pays the label for each artist it wants to list on its store. Since Chinook makes money the more popular an album/artist/genre is, we've been been tasked with figuring out which albums from this record label to add to the Chinook store first. We can pick three of the four artists below to list:

|Artist name|Genre|
|---|---|
|Regal|Hip-Hop|
|Red Tone|Punk|
|Meteor and the Girls|Pop|
|Slim Jim Bites|Blues|

This label specializes in artists from the USA and we currently have none of their artists on the store, so we'll answer this question by figuring out the most popular genre in the USA. To do this, we will write a query that returns the number of tracks each genre sells in the USA by absolute number and by percentage.

In [4]:
%%sql
SELECT
    g."name",
    SUM(il."quantity") no_of_tracks_sold,
    CAST(SUM(il."quantity") AS FLOAT)/(
                                        SELECT
                                            CAST(SUM("quantity") AS FLOAT)
                                        FROM invoice_line
    ) * 100 percentage_of_tracks_sold
FROM invoice_line il
INNER JOIN
    (
        SELECT
            "track_id",
            "genre_id"
        FROM track
    ) t ON t.track_id = il.track_id
INNER JOIN
    (
        SELECT
            "genre_id",
            "name"
        FROM genre
    ) g ON g.genre_id = t.genre_id
INNER JOIN
    (
        SELECT
            "invoice_id",
            "billing_country"
        FROM invoice
    ) i ON i.invoice_id = il.invoice_id
WHERE i."billing_country" = "USA"
GROUP BY g."name"
ORDER BY SUM(il."quantity") DESC

 * sqlite:///chinook.db
Done.


name,no_of_tracks_sold,percentage_of_tracks_sold
Rock,561,11.79314694134959
Alternative & Punk,130,2.732814799243221
Metal,124,2.606684885431995
R&B/Soul,53,1.1141475719991591
Blues,36,0.7567794828673533
Alternative,35,0.7357578305654824
Pop,22,0.46247635064116044
Latin,22,0.46247635064116044
Hip Hop/Rap,20,0.4204330460374185
Jazz,14,0.294303132226193


The results show that of the four albums and corresponding genres that could be added to the Chinook store, Red Tone, Meteor and the Girls, and Slim Jim bites--who sing Punk, Pop, and Blues respectively--would likely perform the best in the USA, since those genres are the most popular 3 of the 4.

### 2.2 - Which employees have the best sales performance?
It's compensation season! Chinook has decided to give its employees holiday bonuses proportional to their sales performances. As a result we need to understand the sales performance of each employee.

To answer this question, we'll be looking at the total dollar amount of sales associated with each sales support agent.

In [5]:
%%sql
CREATE VIEW master_table AS        
    SELECT
        *
    FROM invoice i
    INNER JOIN
        (
            SELECT
                "customer_id",
                "support_rep_id"
            FROM customer
        ) c ON c."customer_id" = i."customer_id"
    INNER JOIN
        (
            SELECT
                "employee_id",
                "title",
                "reports_to",
                "hire_date"
            FROM employee
        ) e ON e."employee_id" = c."support_rep_id"
    WHERE e."title" = "Sales Support Agent"
    ;
    
SELECT
    "employee_id",
    SUM("total") total_sales
FROM master_table
GROUP BY "employee_id"
    
--invoice (customer_id) -> customer (support_rep_id) -> (employee_id) employee

 * sqlite:///chinook.db
(sqlite3.OperationalError) table master_table already exists
[SQL: CREATE VIEW master_table AS        
    SELECT
        *
    FROM invoice i
    INNER JOIN
        (
            SELECT
                "customer_id",
                "support_rep_id"
            FROM customer
        ) c ON c."customer_id" = i."customer_id"
    INNER JOIN
        (
            SELECT
                "employee_id",
                "title",
                "reports_to",
                "hire_date"
            FROM employee
        ) e ON e."employee_id" = c."support_rep_id"
    WHERE e."title" = "Sales Support Agent"
    ;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


Looks like if we went with just the total dollar amount, employee 3 outperforms employee 4, who outperforms employee 5 (I know, shocking that such a company has only 3 sales support agents). The difference in their performances aren't *that* noticeable though--there could also be other factors contributing to this discrepancy. Let's take a look, for example, at when each of these employees were hired.

In [6]:
%%sql
SELECT
    "employee_id",
    SUM("total") total_sales,
    "hire_date"
FROM master_table
GROUP BY "employee_id";

 * sqlite:///chinook.db
Done.


employee_id,total_sales,hire_date
3,1731.5100000000039,2017-04-01 00:00:00
4,1584.0000000000034,2017-05-03 00:00:00
5,1393.920000000002,2017-10-17 00:00:00


Looks like the hunch was right--as we see above, employees who have been at the company longer tend to have higher total sales. This likely plays a role in the difference in performance.

Let's take a look at the performance of employees, using # of customer_id's attached to their names as a metric.

In [7]:
%%sql
SELECT
    "employee_id",
    COUNT(DISTINCT("customer_id")) no_of_customers,
    "hire_date"
FROM master_table
GROUP BY "employee_id"

 * sqlite:///chinook.db
Done.


employee_id,no_of_customers,hire_date
3,21,2017-04-01 00:00:00
4,20,2017-05-03 00:00:00
5,18,2017-10-17 00:00:00


Same story here. Employees who have been at the company longer also have more clients.
I think it's overall hard to compare the employees because the hire date is a confounding variable. **A possible next step** would be to normalize the `total_sales` and `no_of_customers` by the number of months an employee has been at Chinook.

Let's get rid of the view we created earlier, for cleanliness.

In [8]:
%%sql
DROP VIEW master_table

 * sqlite:///chinook.db
Done.


[]

### 2.3 - Analyzing sales data by country.
Chinook management has asked us for an overview of the business. In particular, they have requested the following information:
* Total number of customers
* Total values of sales
* Average value of sales per customer
* Average order value

Additionally, for ease of information digestion, we're going to make it so that any countries for which there is only one customer is aggregated under the country name "Other", and forced to the bottom of the output.

Let's create a view that we can use to answer all these questions, making our downstream work easier.

In [9]:
%%sql
CREATE VIEW master_sales_table AS
    SELECT
        c."customer_id",
        c."country",
        i."total"
    FROM customer c
    INNER JOIN
        (
            SELECT
                "customer_id",
                "total"
            FROM invoice
        ) i ON i."customer_id" = c."customer_id"
    ;
    
SELECT * FROM master_sales_table
LIMIT 5

 * sqlite:///chinook.db
Done.
Done.


customer_id,country,total
18,USA,15.84
30,Canada,9.9
40,France,1.98
18,USA,7.92
27,USA,16.83


#### 2.3.1 - Total number of customers by country
Now let's find the total number of customers, grouped by country. For ease of digestion, we have bucketed all countries with only 1 customer into the category `other`.

In [10]:
%%sql
SELECT
    "country",
    "no_of_customers"
FROM
    (
        SELECT
            *,
            CASE
                WHEN "country" = "other" THEN 1
                ELSE 0
            END AS sort_col
        FROM
            (
                SELECT
                    "country",
                    COUNT(DISTINCT("customer_id")) "no_of_customers"
                FROM master_sales_table
                GROUP BY "country"
                HAVING "no_of_customers" > 1
            )

        UNION

        SELECT
            *,
            CASE
                WHEN "country" = "other" THEN 1
                ELSE 0
            END AS sort_col
        FROM
            (
                SELECT
                    "other" country,
                    SUM("no_of_customers") "no_of_customers"
                FROM 
                    (
                        SELECT
                            "country",
                            COUNT(DISTINCT("customer_id")) "no_of_customers"
                        FROM master_sales_table
                        GROUP BY "country"
                        HAVING "no_of_customers" = 1
                    )
            )
    )
ORDER BY sort_col

 * sqlite:///chinook.db
Done.


country,no_of_customers
Brazil,5
Canada,8
Czech Republic,2
France,5
Germany,4
India,2
Portugal,2
USA,13
United Kingdom,3
other,15


Looks like Chinook has quite the presence in the USA, while countries like the Czech Republic, India, Portugal, and 15 others are pulling up the rear.

#### 2.3.2 - Total value of sales by country

We looked at customers in each country earlier, now let's find the total value of sales by country to tease out just how much money Chinook is making from each country.

In [13]:
%%sql
SELECT
    "country",
    "total"
FROM
    (
        SELECT
            *,
            CASE
                WHEN "country" = "other" THEN 1
                ELSE 0
            END AS sort_col
        FROM
            (
                SELECT
                    "country",
                    "total"
                FROM
                    (
                        SELECT
                            "country",
                            COUNT(DISTINCT("customer_id")) "no_of_customers",
                            SUM("total") "total"
                        FROM master_sales_table
                        GROUP BY "country"
                        HAVING "no_of_customers" > 1
                    )

                UNION

                SELECT
                    "other" "country",
                    SUM("total")
                FROM
                    (
                        SELECT
                            "country",
                            COUNT(DISTINCT("customer_id")) "no_of_customers",
                            SUM("total") "total"
                        FROM master_sales_table
                        GROUP BY "country"
                        HAVING "no_of_customers" = 1
                    )
            )
            ORDER BY sort_col
    )

 * sqlite:///chinook.db
Done.


country,total
Brazil,427.68000000000006
Canada,535.5900000000001
Czech Republic,273.24000000000007
France,389.0699999999999
Germany,334.62
India,183.14999999999998
Portugal,185.13000000000002
USA,1040.4899999999998
United Kingdom,245.52
other,1094.9399999999998


Looks like Chinook has struggling sales in Portugal, India, and the UK to name a few. This could either be because there are few but high-paying customers in these countries, or there are many but low-paying customers in these countries (or somewhere in between). Let's take a closer look on how the number of customers affect these figures, by looking at the average value of sales per customer.

#### 2.3.3 Average value of sales per customer

The Chinook marketing team has been tasked with creating a marketing campaign for countries with low total sales per customer. Management has asked us to identify these countries, so the marketing campaigns can be deployed correctly. Let's take a look.

In [14]:
%%sql
SELECT
    "country",
    CAST("total" AS FLOAT)/CAST("no_of_customers" AS FLOAT) "avg_sales_per_customer"
FROM
    (
        SELECT
            "country",
            SUM("total") "total",
            COUNT(DISTINCT("customer_id")) "no_of_customers"
        FROM
        (
            SELECT
                "customer_id",
                "country",
                SUM("total") total
            FROM master_sales_table
            GROUP BY "customer_id"
        )
        GROUP BY "country"
        HAVING "no_of_customers" > 1
    )
    
UNION

SELECT
    "other" "country",
    AVG("total") "avg_sales_per_customer"
FROM
    (
        SELECT
            "country",
            SUM("total") "total",
            COUNT(DISTINCT("customer_id")) "no_of_customers"
        FROM
        (
            SELECT
                "customer_id",
                "country",
                SUM("total") total
            FROM master_sales_table
            GROUP BY "customer_id"
        )
        GROUP BY "country"
        HAVING "no_of_customers" = 1
    )

 * sqlite:///chinook.db
Done.


country,avg_sales_per_customer
Brazil,85.53599999999999
Canada,66.94875
Czech Republic,136.62
France,77.814
Germany,83.655
India,91.57499999999999
Portugal,92.565
USA,80.03769230769231
United Kingdom,81.83999999999999
other,72.996


It seems like there are opportunities for increased marketing attention in Canada, France, Germany, and the "other" countries.

#### 2.3.4 - Average order value

Let's take a look at the size of the average order value by country.

In [15]:
%%sql
SELECT *
FROM (
    SELECT
        "country",
        "avg_order_value"
    FROM
        (
            SELECT
                "country",
                AVG("total") "avg_order_value",
                COUNT(DISTINCT("customer_id")) "no_of_customers"
            FROM master_sales_table
            GROUP BY "country"
            HAVING "no_of_customers" > 1
        )

    UNION

    SELECT
        "other" "country",
        AVG("avg_order_value") "avg_order_value"
    FROM
        (
            SELECT
                "country",
                AVG("total") "avg_order_value",
                COUNT(DISTINCT("customer_id")) "no_of_customers"
            FROM master_sales_table
            GROUP BY "country"
            HAVING "no_of_customers" = 1
        )
)
ORDER BY "avg_order_value" DESC

 * sqlite:///chinook.db
Done.


country,avg_order_value
Czech Republic,9.108000000000002
United Kingdom,8.768571428571429
India,8.72142857142857
Germany,8.161463414634147
USA,7.942671755725189
France,7.781399999999998
other,7.445071062271063
Canada,7.047236842105265
Brazil,7.0111475409836075
Portugal,6.383793103448276


The three countries with the lowest order values are Canada, Brazil, and Portugal. Probably worth a deeper look as to what can be done from a marketing side to boost these numbers.

In [16]:
%%sql
DROP VIEW master_sales_table

 * sqlite:///chinook.db
Done.


[]

### 2.4 - Categorizing each invoice as an album purchase or not
Chinook management team is evaluating a purchase strategy proposal that would allow them to purchase specific tracks from labels, as opposed to entire albums. This looks attractive to management because they would save money if they didn't have to purchase entire albums, but purchased only the popular songs instead. To examine this proposal, they've asked to analyze how customers currently purchase music from the store--do they purchase more often by album, or by individual tracks?

We can categorize each invoice into two buckets:
* Whole albums purchases
* Individual track purchases

We know, for the purposes of this analysis, that invoices with a whole album purchased ***do not*** also include individual tracks. In other words, an invoice is either entirely a whole album, or entirely a collection of tracks from different albums. We can use this information to aid our analysis by creating a view that comprises of one (effectively random) track from each invoice. We can then find the other `track_id`s in the album that this track belongs to, and cross reference that list with the other `track_id`s in the invoice.

In [17]:
%%sql
CREATE VIEW invoice_single_track_table AS
    SELECT
        "invoice_id",
        MAX("track_id") "track_id"
    FROM invoice_line
    GROUP BY "invoice_id"
    ;

 * sqlite:///chinook.db
Done.


[]

Ensure that the view was created properly...

In [18]:
%%sql
SELECT * FROM invoice_single_track_table
LIMIT 5

 * sqlite:///chinook.db
Done.


invoice_id,track_id
1,1173
2,3476
3,2646
4,3448
5,2002


Now we find the other `track_id`s in the album that this track belongs to, and cross reference that list with the other `track_id`s in the invoice, and finally compile the results.

In [19]:
%%sql
SELECT
    COUNT(*) "no_of_invoices",
    CAST(SUM("album_purchase?") AS FLOAT)/CAST(COUNT(*) AS FLOAT) * 100 "pct_album_purchase"
FROM
(
    SELECT
        "invoice_id",
        CASE
            WHEN
            (
                SELECT
                    "track_id"
                FROM track
                WHERE "album_id" = (
                                    SELECT
                                        "album_id"
                                    FROM track
                                    WHERE "track_id" = istt."track_id"
                                    )

                EXCEPT

                SELECT
                    "track_id"
                FROM invoice_line il
                WHERE "invoice_id" = istt."invoice_id"
            ) IS NULL
            AND
            (
                SELECT
                    "track_id"
                FROM invoice_line il
                WHERE "invoice_id" = istt."invoice_id"

                EXCEPT

                SELECT
                    "track_id"
                FROM track
                WHERE "album_id" = (
                                    SELECT
                                        "album_id"
                                    FROM track
                                    WHERE "track_id" = istt."track_id"
                                    )
            ) IS NULL
            THEN 1
            ELSE 0
        END AS "album_purchase?"
    FROM invoice_single_track_table istt
)

 * sqlite:///chinook.db
Done.


no_of_invoices,pct_album_purchase
614,18.566775244299674


Looks like of all the invoices, only 18% of them are full album purchases, indicating that most customers of the Chinook music store pick and choose the individual tracks they like, when shopping. This indicates that the Chinook management team might be better served purchasing only the popular songs from the record companies. As a next step, it would be worth exploring just how this strategy would affect Chinook's customers. In other words, if Chinook management only bought the most popular songs for, say, the past two years, how much revenue would they have missed out on in terms of song purchases? Then we can compare this number to the amount of money they'd have saved by not purchasing the full albums. If the latter number is greater than the former, then the management team should only buy popular songs, instead of entire albums.

In [20]:
%%sql
DROP VIEW invoice_single_track_table

 * sqlite:///chinook.db
Done.


[]

In this project, we've used SQL to explore some business and personnel questions for the hypothetical online music store, Chinook Inc. We joined several tables, aggregated multiple statistics, and even segmented the data by some specified rules (like album purchase invoices vs single song purchase invoices). 